# End-to-end Ensemble

# 1. Feature Engineering

In [30]:
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error,r2_score
import time
%matplotlib inline

import xgboost
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor


from bayes_opt import BayesianOptimization

import warnings
warnings.filterwarnings("ignore")

In [2]:
"""
Contributions from:
DSEverything - Mean Mix - Math, Geo, Harmonic (LB 0.493) 
https://www.kaggle.com/dongxu027/mean-mix-math-geo-harmonic-lb-0-493
JdPaletto - Surprised Yet? - Part2 - (LB: 0.503)
https://www.kaggle.com/jdpaletto/surprised-yet-part2-lb-0-503
hklee - weighted mean comparisons, LB 0.497, 1ST
https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st

Also all comments for changes, encouragement, and forked scripts rock

Keep the Surprise Going
"""

import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime

data = {
    'tra': pd.read_csv('../../../mltestdata/05_recruit/air_visit_data.csv'),
    'as': pd.read_csv('../../../mltestdata/05_recruit/air_store_info.csv'),
    'hs': pd.read_csv('../../../mltestdata/05_recruit/hpg_store_info.csv'),
    'ar': pd.read_csv('../../../mltestdata/05_recruit/air_reserve.csv'),
    'hr': pd.read_csv('../../../mltestdata/05_recruit/hpg_reserve.csv'),
    'id': pd.read_csv('../../../mltestdata/05_recruit/store_id_relation.csv'),
    'tes': pd.read_csv('../../../mltestdata/05_recruit/sample_submission.csv'),
    'hol': pd.read_csv('../../../mltestdata/05_recruit/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

#sure it can be compressed...
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
# NEW FEATURES FROM Georgii Vyshnia
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

# NEW FEATURES FROM JMBULL
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

# NEW FEATURES FROM Georgii Vyshnia
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

for c, dtype in zip(train.columns, train.dtypes):	
    if dtype == np.float64:		
        train[c] = train[c].astype(np.float32)

for c, dtype in zip(test.columns, test.dtypes):	
    if dtype == np.float64:		
        test[c] = test[c].astype(np.float32)

combine = [train, test]
gw_list = ['2016-04-29','2016-04-30','2016-05-01','2016-05-02','2016-05-03','2016-05-04','2016-05-05','2017-04-29','2017-04-30','2017-05-01','2017-05-02','2017-05-03','2017-05-04','2017-05-05']
post_gw_list=['2016-05-06']
train['gw_flg'] = 0
train['post_gw_flg'] = 0
test['gw_flg'] = 0
test['post_gw_flg'] = 0
update_gw_list = [["0" for i in range(3)] for j in range(len(gw_list))]
update_post_gw_list = [["0" for i in range(3)] for j in range(len(post_gw_list))]

from datetime import date
for index, gw_date in enumerate(gw_list):
    temp_list = gw_date.split("-")
    for col_i, temp_figure in enumerate(temp_list):
        update_gw_list[index][col_i]=int(temp_figure)
        
    #print("{}  {}  {}".format(update_list[index][0],update_list[index][1],update_list[index][2]))
    
for index, gw_date in enumerate(post_gw_list):
    temp_list = gw_date.split("-")
    for col_i, temp_figure in enumerate(temp_list):
        update_post_gw_list[index][col_i]=int(temp_figure)

for dataset in combine:
    for index in range(len(update_gw_list)):
        dataset.loc[dataset.visit_date == date(update_gw_list[index][0],update_gw_list[index][1],update_gw_list[index][2]), 'gw_flg'] = 1
        
for dataset in combine:
    for index in range(len(update_post_gw_list)):
        dataset.loc[dataset.visit_date == date(update_post_gw_list[index][0],update_post_gw_list[index][1],update_post_gw_list[index][2]), 'post_gw_flg'] = 1     

In [3]:
drop_cols=['visitors','air_store_id','visit_date','id']
y_train=train['visitors']
x_train=train.drop(drop_cols, axis=1)

x_test=test.copy()
x_test=x_test.drop(drop_cols, axis=1)

# 2. K-fold CV with Out-of-Fold Prediction

__Define Utility Function__

In [4]:
#Define a evaluation function

def rmsle(preds, true):
    rmsle = np.sqrt(mean_squared_error(np.log1p(true), np.log1p(preds)))
    return float(rmsle)

def eval_rmsle(y, pred):
    return mean_squared_error(y, pred)**0.5

In [5]:
#Define a evaluation matrix 
from sklearn.metrics.scorer import make_scorer
RMSLE = make_scorer(rmsle)

In [10]:
kf=StratifiedKFold(n_splits=5, shuffle=True, random_state=2018)

### 2.1.1 Sklearn K-fold & OOF function

In [11]:
### Modified cross-validation function copied from Yifan's cool kernel. 
### Source: https://www.kaggle.com/yifanxie/porto-seguro-tutorial-end-to-end-ensemble

def cross_validate_sklearn(clf, x_train, y_train, x_test, kf, scale=False, verbose=True):
    start_time=time.time()
    
    # initialise the size of out-of-fold train an test prediction
    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))

    # use the kfold object to generate the required folds
    for i, (train_index, test_index) in enumerate(kf.split(x_train, y_train)):
        # generate training folds and validation fold
        x_train_kf, x_val_kf = x_train.loc[train_index, :], x_train.loc[test_index, :]
        y_train_kf, y_val_kf = y_train[train_index], y_train[test_index]

        # perform scaling if required i.e. for linear algorithms
        if scale:
            scaler = StandardScaler().fit(x_train_kf.values)
            x_train_kf_values = scaler.transform(x_train_kf.values)
            x_val_kf_values = scaler.transform(x_val_kf.values)
            x_test_values = scaler.transform(x_test.values)
        else:
            x_train_kf_values = x_train_kf.values
            x_val_kf_values = x_val_kf.values
            x_test_values = x_test.values
        
        clf.fit(x_train_kf_values, np.log1p(y_train_kf.values))

        val_pred=np.expm1(clf.predict(x_val_kf_values))

        train_pred[test_index] += val_pred

        y_test_preds = np.expm1(clf.predict(x_test_values))
        test_pred += y_test_preds

        fold_rmsle = eval_rmsle(np.log1p(y_val_kf.values), np.log1p(val_pred))

        if verbose:
            #print('fold cv {} AUC score is {:.6f}, Gini_Norm score is {:.6f}'.format(i, fold_auc, fold_gini_norm))
            print('fold cv {} RMSLE score is {:.6f}'.format(i, fold_rmsle))

    test_pred /= kf.n_splits

    cv_rmsle = eval_rmsle(np.log1p(y_train), np.log1p(train_pred))
        
    #cv_score = [cv_auc, cv_gini_norm]
    cv_score = [cv_rmsle]
    if verbose:

        print('cv RMSLE score is {:.6f}'.format(cv_rmsle))
        end_time = time.time()
        print("it takes %.3f seconds to perform cross validation" % (end_time - start_time))
    return cv_score, train_pred, test_pred


### 2.1.2 Xgboost K-fold & OOF function

In [38]:
def cross_validate_xgb(params, x_train, y_train, x_test, kf,  verbose=True, verbose_eval=50, scoreonly=False):
    start_time=time.time()
    nround=[]
    # the prediction matrix need to contains 3 columns, one for the probability of each class
    #train_pred = np.zeros((x_train.shape[0],3))
    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))
    
    # self-defined eval metric
    # f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
    # binary error
    def feval_rmsle(preds, train_data):
        preds = np.expm1(preds)
        true = np.expm1(train_data.get_label())
        #return 'rmsle', rmsle(true, preds), False

        return 'rmsle', rmsle(preds, true), False

    # use the k-fold object to enumerate indexes for each training and validation fold
    for i, (train_index, val_index) in enumerate(kf.split(x_train, y_train)):
        x_train_kf, x_val_kf = x_train.loc[train_index, :], x_train.loc[val_index, :]

        #y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
        y_train_kf, y_val_kf = np.log1p(y_train[train_index]), np.log1p(y_train[val_index])
        x_test_kf=x_test.copy()
        
        d_train = xgboost.DMatrix(x_train_kf, y_train_kf)
        d_val=xgboost.DMatrix(x_val_kf, y_val_kf)
        d_test = xgboost.DMatrix(x_test_kf)
        
        watchlist= [(d_train, "train"), (d_val, 'val')]
        bst = xgboost.train(params=params, 
                            dtrain=d_train, 
                            num_boost_round=8000, 
                            early_stopping_rounds=100,
                            evals=watchlist, 
                            verbose_eval=verbose_eval)        
        
#        y_val_kf_preds=bst.predict(d_val, ntree_limit=bst.best_ntree_limit)
        y_val_kf_preds=np.expm1(bst.predict(d_val, ntree_limit=bst.best_ntree_limit))
        nround.append(bst.best_ntree_limit)
        
        train_pred[val_index] += y_val_kf_preds
#        test_pred += np.expm1((bst.predict(x_test, ntree_limit=bst.best_ntree_limit)))
        test_pred += np.expm1(bst.predict(d_test))
        
        
        #fold_cv = log_loss(y_val_kf.values, y_val_kf_preds)
        #fold_rmsle = rmsle(np.expm1(train_pred[val_index]),np.expm1(y_val_kf.values))
        fold_rmsle = rmsle(train_pred[val_index],np.expm1(y_val_kf.values))
        fold_cv = fold_rmsle
        
        if verbose:
            print('fold cv {} rmsle score is {:.6f}'.format(i, fold_cv))

    test_pred = test_pred / kf.n_splits
    #cv_score = log_loss(y_train, train_pred)
    #cv_score = rmsle(np.expm1(train_pred), y_train)
    cv_score = rmsle(train_pred, y_train)
    
    if verbose:
        print('cv rmsle score is {:.6f}'.format(cv_score))    
        end_time = time.time()
        print("it takes %.3f seconds to perform cross validation" % (end_time - start_time))
 
    if scoreonly:
        #return cv_score # for the purpose of bayesian optimisation, we only need to return the CV score
        return cv_score
    else:
        return (cv_score,train_pred,test_pred)

### 2.1.3 LigthGBM K-fold & OOF function

In [13]:
def cross_validate_lgb(params, x_train, y_train, x_test, 
                        kf, 
                        cat_features=[],
                        verbose=True, verbose_eval=100, nseeds=1, df_input=True,
                        early_stopping=100, num_boost_round=8000, scoreonly=False):

    start_time = time.time()
    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))

    # self-defined eval metric
    # f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
    # binary error
    def feval_rmsle(preds, train_data):
        preds = np.expm1(preds)
        true = np.expm1(train_data.get_label())

        return 'rmsle', rmsle(preds, true), False
       
    if len(cat_features)==0: use_cat=False

    # use the k-fold object to enumerate indexes for each training and validation fold
    for i, (train_index, val_index) in enumerate(kf.split(x_train, y_train)): # folds 1, 2 ,3 ,4, 5
        # example: training from 1,2,3,4; validation from 5
        if df_input:
            x_train_kf, x_val_kf = x_train.loc[train_index, :], x_train.loc[val_index, :]
        else:
            x_train_kf, x_val_kf = x_train[train_index], x_train[val_index]

        y_train_kf, y_val_kf = np.log1p(y_train[train_index]), np.log1p(y_train[val_index])

        for seed in range(nseeds):
            params['feature_fraction_seed'] = seed
            params['bagging_seed'] = seed

            if use_cat:
                lgb_train = lgb.Dataset(x_train_kf, y_train_kf, categorical_feature=cat_features)
                lgb_val = lgb.Dataset(x_val_kf, y_val_kf, reference=lgb_train, categorical_feature=cat_features)

            else:
                lgb_train = lgb.Dataset(x_train_kf, y_train_kf)
                lgb_val = lgb.Dataset(x_val_kf, y_val_kf, reference=lgb_train)

            gbm = lgb.train(params,
                            lgb_train,
                            num_boost_round=num_boost_round,
                            valid_sets=[lgb_val],
                            early_stopping_rounds=early_stopping,
                            feval=feval_rmsle,
                            verbose_eval=verbose_eval)

            val_pred = np.expm1(gbm.predict(x_val_kf, num_iteration=gbm.best_iteration))
            
            train_pred[val_index] += val_pred
            test_pred += np.expm1((gbm.predict(x_test, num_iteration=gbm.best_iteration)))

                                
        train_pred[val_index] = val_pred/nseeds

        #fold_rmsle = rmsle(np.expm1(y_val_kf.values), train_pred[val_index])
        fold_rmsle = rmsle(train_pred[val_index],np.expm1(y_val_kf.values))
        if verbose:
            print('fold cv {} RMSLE score is {:.6f}'.format(i, fold_rmsle))

    test_pred = test_pred / (nseeds * kf.n_splits)
    #cv_score = rmsle(y_train, train_pred)
    cv_score = rmsle(train_pred, np.expm1(y_train))
    
    if verbose:
        print('cv RMSLE score is {:.6f}'.format(cv_score))
        end_time = time.time()
        print("it takes %.3f seconds to perform cross validation" % (end_time - start_time))
    #return cv_score, np.expm1(train_pred),test_pred
    
    if scoreonly:
        return cv_score
    else:
        return cv_score, train_pred, test_pred

# 3. Level 2 ensemble

## Data import

### Level 1   GBM

In [14]:
#learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, max_depth =10　=> around 0.5
params = {'learning_rate': 0.1, 
        'n_estimators': 375,
         'max_depth' : 10,
         'min_samples_leaf': 3}
model1 = ensemble.GradientBoostingRegressor(**params)

In [15]:
outcomes =cross_validate_sklearn(model1, x_train, y_train ,x_test, kf, scale=False, verbose=True)

fold cv 0 RMSLE score is 0.491316
fold cv 1 RMSLE score is 0.486795
fold cv 2 RMSLE score is 0.483089
fold cv 3 RMSLE score is 0.484583
fold cv 4 RMSLE score is 0.483190
cv RMSLE score is 20.703650
it takes 6124.088 seconds to perform cross validation


In [16]:
print("Start...\n")

lv1_gbm_train_pred=outcomes[1]
lv1_gbm_test_pred=outcomes[2]

lv1_gbm_train_pred=pd.DataFrame(columns=['visitors'], data=lv1_gbm_train_pred)
lv1_gbm_test_pred=pd.DataFrame(columns=['visitors'], data=lv1_gbm_test_pred)

lv1_gbm_train_pred.to_csv('lv1_gbm_train_pred.csv', index=False)
lv1_gbm_test_pred.to_csv('lv1_gbm_test_pred.csv', index=False)

print("Finished.")

Start...

Finished.


### Others

In [17]:
lv1_lgb_train_pred = pd.read_csv('./lv1_lgb_train_pred.csv')
lv1_lgb_test_pred = pd.read_csv('./lv1_lgb_test_pred.csv')

lv1_knr_train_pred = pd.read_csv('./lv1_knr_train_pred.csv')
lv1_knr_test_pred = pd.read_csv('./lv1_knr_test_pred.csv')

lv1_xgb_train_pred = pd.read_csv('./lv1_xgb_train_pred.csv')
lv1_xgb_test_pred = pd.read_csv('./lv1_xgb_test_pred.csv')

lv1_rfr_train_pred = pd.read_csv('./lv1_rfr_train_pred.csv')
lv1_rfr_test_pred = pd.read_csv('./lv1_rfr_test_pred.csv')

## Generate L1 output dataframe

In [18]:
columns=['gbm','lgb','knr','xgb','rfr']
train_pred_df_list=[lv1_gbm_train_pred, lv1_lgb_train_pred, lv1_knr_train_pred, lv1_xgb_train_pred, lv1_rfr_train_pred]

test_pred_df_list=[lv1_gbm_test_pred, lv1_lgb_test_pred, lv1_knr_test_pred, lv1_xgb_test_pred, lv1_rfr_test_pred]

lv1_train_df=pd.DataFrame(columns=columns)
lv1_test_df=pd.DataFrame(columns=columns)

In [19]:
for i in range(0,len(columns)):
    lv1_train_df[columns[i]]=train_pred_df_list[i]['visitors']
    lv1_test_df[columns[i]]=test_pred_df_list[i]['visitors']

In [20]:
lv1_train_df.head()

,gbm,lgb,knr,xgb,rfr
0,17.169498,20.186493,19.627612,10.031622,20.851900
1,16.308033,19.040413,18.991099,10.914748,18.261963
2,27.069372,24.524543,23.468845,11.513679,30.183405
3,21.419504,23.505738,17.624440,11.901163,21.146152
4,13.882360,10.796744,13.756800,23.741966,13.492072


In [21]:
lv1_test_df.head()

,gbm,lgb,knr,xgb,rfr
0,1.850958,4.927308,6.199765,5.034050,2.481164
1,22.194486,20.985203,21.752924,21.175813,20.729363
2,28.778062,26.198808,26.588196,26.141714,24.485625
3,28.267724,29.470553,27.608413,30.093324,24.867540
4,30.377499,31.224496,26.794744,32.709636,27.772946


## 4.1 Level 2 LGBM

In [22]:
lgb_params = {
    'num_leaves': int(242.70662305751688), 
    'max_depth': int(48.434894524736237), 
    'learning_rate': 0.22739737221785333, 
    'scale_pos_weight': 4449.6905120668471, 
    'min_sum_hessian_in_leaf': 2.9535851736769945, 
    'subsample': 0.78319355511875155, 
    'colsample_bytree': 0.60980110433768919, 
    'feature_fraction': 0.38725837624747139, 
    'bagging_fraction': 0.25993005950069908, 
    'bagging_freq': int(0.0063304070505429966), 
    'lambda_l1': 0.10468672203086582, 
    'lambda_l2': 0.73311141753276088, 
    'n_estimators': 7.574050255399829, 
    'reg_lambda': 0.7867690387142483, 
    'min_gain_to_split': 0.32711184887807354} 

In [27]:
lgb_lv2_outcomes=cross_validate_lgb(lgb_params,lv1_train_df, y_train ,lv1_test_df,kf, verbose_eval=False)

lgb_lv2_cv=lgb_lv2_outcomes[0]
lgb_lv2_train_pred=lgb_lv2_outcomes[1]
lgb_lv2_test_pred=lgb_lv2_outcomes[2]

fold cv 0 RMSLE score is 0.477037
fold cv 1 RMSLE score is 0.472756
fold cv 2 RMSLE score is 0.469766
fold cv 3 RMSLE score is 0.470899
fold cv 4 RMSLE score is 0.469273
cv RMSLE score is 0.471958
it takes 51.440 seconds to perform cross validation


In [28]:
lgb_lv2_train_pred_df=pd.DataFrame(columns=['visitors'], data=lgb_lv2_train_pred)
lgb_lv2_test_pred_df=pd.DataFrame(columns=['visitors'], data=lgb_lv2_test_pred)

lgb_lv2_train_pred_df.to_csv('lv2_lgb_train_pred.csv', index=False)
lgb_lv2_test_pred_df.to_csv('lv2_lgb_test_pred.csv', index=False)

## 4.2 Level 2 Random Forest

In [34]:
rf_lv2=RandomForestRegressor(max_depth=5,
                            n_estimators=500,#100 too low
                            min_samples_leaf=50,
                            min_samples_split=10
                           )

rf_lv2_outcomes = cross_validate_sklearn(rf_lv2, lv1_train_df, y_train ,lv1_test_df, kf, 
                                            scale=False, verbose=True)
rf_lv2_cv=rf_lv2_outcomes[0]
rf_lv2_train_pred=rf_lv2_outcomes[1]
rf_lv2_test_pred=rf_lv2_outcomes[2]

fold cv 0 RMSLE score is 0.478081
fold cv 1 RMSLE score is 0.473846
fold cv 2 RMSLE score is 0.470958
fold cv 3 RMSLE score is 0.471976
fold cv 4 RMSLE score is 0.470127
cv RMSLE score is 20.460432
it takes 795.987 seconds to perform cross validation


In [35]:
rf_lv2_train_pred_df=pd.DataFrame(columns=['visitors'], data=rf_lv2_train_pred)
rf_lv2_test_pred_df=pd.DataFrame(columns=['visitors'], data=rf_lv2_test_pred)

rf_lv2_train_pred_df.to_csv('lv2_rf_train_pred.csv', index=False)
rf_lv2_test_pred_df.to_csv('lv2_rf_test_pred.csv', index=False)

## 4.3 Level 2 XGB

In [36]:
xgb_params = {
    'max_depth': int(9.6075334961758525), 
    'learning_rate': 0.12750069121004876, 
    'subsample': 0.70746966313901982, 
    'colsample_bytree': 0.91300804206695696, 
    'gamma': 0.79704305097433081, 
    'min_child_weight': 11.84689258578884, 
    'max_delta_step': 2.4269823914863409, 
    'n_estimators': 24.064145942538754, 
    'min_samples_split': 11.528636630859337, 
    'max_features': 0.74184703574431088
}

In [39]:
print("Starting xgboost...")

xgb_lv2_outcomes=cross_validate_xgb(xgb_params, lv1_train_df, y_train, lv1_test_df, kf)

xgb_lv2_cv=xgb_lv2_outcomes[0]
xgb_lv2_train_pred=xgb_lv2_outcomes[1]
xgb_lv2_test_pred=xgb_lv2_outcomes[2]

xgb_lv2_train_pred_df=pd.DataFrame(columns=['visitors'], data=xgb_lv2_train_pred)
xgb_lv2_test_pred_df=pd.DataFrame(columns=['visitors'], data=xgb_lv2_test_pred)

xgb_lv2_train_pred_df.to_csv('lv2_xgb_train_pred.csv', index=False)
xgb_lv2_test_pred_df.to_csv('lv2_xgb_test_pred.csv', index=False)

print("Finished.")

Starting xgboost...
[0]	train-rmse:2.1746	val-rmse:2.18104
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[50]	train-rmse:0.457013	val-rmse:0.477576
[100]	train-rmse:0.450998	val-rmse:0.478429
Stopping. Best iteration:
[44]	train-rmse:0.457894	val-rmse:0.477501

fold cv 0 rmsle score is 0.477501
[0]	train-rmse:2.17575	val-rmse:2.17647
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[50]	train-rmse:0.458824	val-rmse:0.473939
[100]	train-rmse:0.452679	val-rmse:0.474737
Stopping. Best iteration:
[43]	train-rmse:0.459512	val-rmse:0.47388

fold cv 1 rmsle score is 0.473880
[0]	train-rmse:2.17615	val-rmse:2.17465
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[50]	train-rmse:0.459152	val-rmse:0.470318
[100]	train

# 4. Level 3 ensemble

## Data import

In [41]:
lv2_lgb_train_pred = pd.read_csv('./lv2_lgb_train_pred.csv')
lv2_lgb_test_pred = pd.read_csv('./lv2_lgb_test_pred.csv')

lv2_xgb_train_pred = pd.read_csv('./lv2_xgb_train_pred.csv')
lv2_xgb_test_pred = pd.read_csv('./lv2_xgb_test_pred.csv')

lv2_rfr_train_pred = pd.read_csv('./lv2_rf_train_pred.csv')
lv2_rfr_test_pred = pd.read_csv('./lv2_rf_test_pred.csv')

## 4.1 Generate L2 output dataframe

In [45]:
columns=['lgb','xgb','rfr']
train_pred_df_list=[lv1_lgb_train_pred, lv1_xgb_train_pred, lv1_rfr_train_pred]

test_pred_df_list=[lv1_lgb_test_pred, lv1_xgb_test_pred, lv1_rfr_test_pred]

lv2_train_df=pd.DataFrame(columns=columns)
lv2_test_df=pd.DataFrame(columns=columns)

In [46]:
for i in range(0,len(columns)):
    lv2_train_df[columns[i]]=train_pred_df_list[i]['visitors']
    lv2_test_df[columns[i]]=test_pred_df_list[i]['visitors']

## 4.2 Level 3 XGB 

In [48]:
print("Starting xgboost...")

xgb_lv3_outcomes=cross_validate_xgb(xgb_params, lv2_train_df, y_train, lv2_test_df, kf)

xgb_lv3_cv=xgb_lv3_outcomes[0]
xgb_lv3_train_pred=xgb_lv3_outcomes[1]
xgb_lv3_test_pred=xgb_lv3_outcomes[2]

xgb_lv3_train_pred_df=pd.DataFrame(columns=['visitors'], data=xgb_lv3_train_pred)
xgb_lv3_test_pred_df=pd.DataFrame(columns=['visitors'], data=xgb_lv3_test_pred)

xgb_lv3_train_pred_df.to_csv('lv3_xgb_train_pred.csv', index=False)
xgb_lv3_test_pred_df.to_csv('lv3_xgb_test_pred.csv', index=False)

print("Finished.")

Starting xgboost...
[0]	train-rmse:2.17438	val-rmse:2.18043
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[50]	train-rmse:0.463203	val-rmse:0.480638
[100]	train-rmse:0.4599	val-rmse:0.48115
[150]	train-rmse:0.457077	val-rmse:0.48153
Stopping. Best iteration:
[50]	train-rmse:0.463203	val-rmse:0.480638

fold cv 0 rmsle score is 0.480638
[0]	train-rmse:2.17572	val-rmse:2.17611
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[50]	train-rmse:0.464863	val-rmse:0.476804
[100]	train-rmse:0.46098	val-rmse:0.477157
Stopping. Best iteration:
[46]	train-rmse:0.465193	val-rmse:0.476779

fold cv 1 rmsle score is 0.476779
[0]	train-rmse:2.17601	val-rmse:2.17422
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[50]	train-rm

__Saving data__

In [49]:
lv3_xgb_train_pred = xgb_lv3_train_pred_df.copy()
lv3_xgb_train_pred.to_csv('lv3_xgb_train_pred.csv', index=False)

lv3_xgb_test_pred = xgb_lv3_test_pred_df.copy()
lv3_xgb_test_pred.to_csv('lv3_xgb_test_pred.csv', index=False)

# Submission

In [51]:
test['visitors'] = xgb_lv2_test_pred_df.values
sub = test[['id','visitors']].copy()
sub1 = sub.copy()

In [52]:
# from hklee
# https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st/code
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):
    pd.read_csv(fn)for fn in glob.glob('../../../mltestdata/05_recruit/*.csv')}

for k, v in dfs.items(): locals()[k] = v

wkend_holidays = date_info.apply(
    (lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  

visit_data = air_visit_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)

wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) # cumbersome, should be better ways.

sample_submission['air_store_id'] = sample_submission.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['calendar_date'] = sample_submission.id.map(lambda x: x.split('_')[2])
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='calendar_date', how='left')
sample_submission = sample_submission.merge(visitors, on=[
    'air_store_id', 'day_of_week', 'holiday_flg'], how='left')

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[visitors.holiday_flg==0], on=('air_store_id', 'day_of_week'), 
    how='left')['visitors_y'].values

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), 
    on='air_store_id', how='left')['visitors_y'].values

sample_submission['visitors'] = sample_submission.visitors.map(pd.np.expm1)
sub2 = sample_submission[['id', 'visitors']].copy()
sub_merge = pd.merge(sub1, sub2, on='id', how='inner')

In [53]:
sub_merge['visitors'] = (sub_merge['visitors_x'] + sub_merge['visitors_y']* 1.1)/2
sub_merge[['id', 'visitors']].to_csv('submission_rs_recruit_v17_stacking_02_01.csv', index=False)

# gw_flag
# xgb
# Bopt
# weight
# Stacking
# Local 0.475844
# PubLB 0.480
# PriLB 0.581